In [1]:
import torch
from torch import nn

torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## Step 1 : Restructuring OS

In [2]:
import os

def restructure(dir, num_classes):
    for i in range(1, num_classes+1):
        dir_to_create = f".\\{dir}\\flower{str(i)}"
        
        if not os.path.isdir(dir_to_create):
            os.mkdir(dir_to_create)

    cwd = os.path.abspath(dir)

    for file in os.listdir(dir):
        if file.endswith(".jpg"):
            os.replace(f"{cwd}\\{file}", f"{cwd}\\flower{file.split("_")[0]}\\{file}")

restructure("train_data", 60)
restructure("val_data", 60)

## Step 2 : Load Dataset into torch Loader

In [3]:
import pandas as pd

fft_train_df = pd.read_csv("./fft_data/image_features_extracted.csv")
fft_val_df = pd.read_csv("./fft_data/image_val_extracted.csv")

fft_X_train, fft_y_train = fft_train_df.loc[:, fft_train_df.columns[:-1]], fft_train_df["label"]
fft_X_val, fft_y_val = fft_val_df.loc[:, fft_val_df.columns[:-1]], fft_val_df["label"]

In [4]:
import torch.utils.data
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import MinMaxScaler

transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Lambda(lambda x: torch.flatten(x))
])

train_fds = datasets.ImageFolder("train_data", transform=transform)
val_fds = datasets.ImageFolder("val_data", transform=transform)

train_fdl = torch.utils.data.DataLoader(train_fds, batch_size=len(train_fds))
val_fdl = torch.utils.data.DataLoader(val_fds, batch_size=len(val_fds))

temp_train = next(iter(train_fdl))
X_train, y_train = temp_train[0].numpy(), temp_train[1]
temp_val = next(iter(val_fdl))
X_val, y_val = temp_val[0].numpy(), temp_val[1]

pca_components = 30
pca = PCA(n_components=pca_components)
X_train = pca.fit_transform(X_train)
X_train = np.concatenate((X_train, fft_X_train.to_numpy()), axis=1).astype("float32")
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

X_val = pca.transform(X_val)
X_val = np.concatenate((X_val, fft_X_val.to_numpy()), axis=1).astype("float32")
X_val = scaler.transform(X_val)

batch_size = 256
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train), y_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val), y_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

In [5]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.496788,0.571908,0.617521,0.343498,0.439246,0.599876,0.147447,0.802232,0.223478,0.205480,...,0.634793,0.535059,0.570032,1.000000,0.005812,0.694217,0.419212,0.656002,0.634794,0.612030
1,0.289778,0.796574,0.578825,0.480469,0.477164,0.562181,0.221664,0.603518,0.437315,0.217546,...,0.360364,0.194557,0.576954,0.934255,0.284599,0.188066,0.475780,0.405281,0.360364,0.439339
2,0.558748,0.510923,0.563075,0.570485,0.395564,0.536151,0.178371,0.602141,0.422621,0.065182,...,0.586309,0.531042,0.461712,0.978210,0.179493,0.542466,0.618227,0.581005,0.586309,0.635975
3,0.447296,0.690135,0.702413,0.452201,0.517696,0.530988,0.135481,0.575690,0.549258,0.232342,...,0.566509,0.611819,0.573005,0.822538,0.040432,0.249732,0.515271,0.590379,0.566509,0.350454
4,0.138099,0.857515,0.593678,0.559746,0.449587,0.542685,0.071387,0.737738,0.427403,0.318712,...,0.250152,0.000000,0.808281,0.997150,0.000000,0.013175,0.495402,0.420650,0.250151,0.160931


## Step 3 : Define Model

In [6]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(pca_components + fft_X_train.shape[1], 100),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(100, 60),
        )

    def forward(self, X):
        logits = self.linear_relu_stack(X)
        return logits
    
model = Model().to(device)
print(model)

Model(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=52, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=100, out_features=60, bias=True)
  )
)


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        one_hot_y = torch.eye(60)[y].to(device)
        y = y.to(device)

        pred = model(X)
        loss = loss_fn(pred, one_hot_y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss, current = loss.item(), batch * batch_size + len(X)
        # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            one_hot_y = torch.eye(60)[y].to(device)
            y = y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, one_hot_y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Step 4 : Execution

In [8]:
import torch.optim

learning_rate = 2e-3
epochs = 200

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(val_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 2.8%, Avg loss: 4.089927 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 4.5%, Avg loss: 4.077950 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 6.5%, Avg loss: 4.052220 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 8.2%, Avg loss: 4.016770 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 7.8%, Avg loss: 3.949007 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 10.0%, Avg loss: 3.870411 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 11.2%, Avg loss: 3.801204 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 11.0%, Avg loss: 3.705123 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 15.0%, Avg loss: 3.619907 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 15.0%, Avg loss: 3.540473 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 16.0%, Avg loss: 3.